In [2]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DecimalType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import LongType
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorIndexer, IndexToString, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

ImportError: cannot import name 'OneHotEncoderEstimator' from 'pyspark.ml.feature' (/opt/anaconda3/lib/python3.7/site-packages/pyspark/ml/feature.py)

In [2]:
sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-0.25.1-cp36-cp36m-manylinux1_x86_64.whl (10.5 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

  Attempting uninstall: numpy
    Found existing installation: numpy 1.14.5
    Not uninstalling numpy at /usr/local/lib64/python3.6/site-packages, outside environment /tmp/1620083632056-0
    Can't uninstall 'numpy'. No files were found to uninstall.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python36-sagemaker-pyspark 1.2.4 requires pyspark==2.3.2, which is not installed.

In [3]:
import pandas as pd
import numpy as np
from pyspark import SparkContext
import pyspark.sql 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
data = sqlContext.read.load('s3://aws-logs-862133200906-us-east-1/elasticmapreduce/telecom_users.csv',format='com.databricks.spark.csv', header = True, inferSchema='true')

NameError: name 'sqlContext' is not defined

In [47]:
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: integer (nullable = true)
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)

In [48]:
data = data.drop('_c0').drop('customerID')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
cols = data.columns
cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

In [50]:
pd.DataFrame(data.take(5), columns = data.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   gender  SeniorCitizen Partner  ... MonthlyCharges  TotalCharges Churn
0    Male              0     Yes  ...          24.10       1734.65    No
1  Female              0      No  ...          88.15        3973.2    No
2  Female              1     Yes  ...          74.95       2869.85   Yes
3    Male              0      No  ...          55.90         238.5    No
4    Male              0      No  ...          53.45         119.5    No

[5 rows x 20 columns]

In [51]:
data.groupby('StreamingMovies').count().toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       StreamingMovies  count
0                   No   2356
1                  Yes   2339
2  No internet service   1291

In [52]:
Categorical_ = [item[0] for item in data.dtypes if item[1].startswith('string')]
Numerical_ = [item[0] for item in data.dtypes if item[1] == 'int' or item[1] == 'double']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
stages=[]
for categoricalCol in Categorical_:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols = [categoricalCol + 'classVec'])
    stages += [stringIndexer, encoder]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
label_stringIdx = StringIndexer(inputCol = 'Churn', outputCol = 'label')
stages += [label_stringIdx]
assemblerInputs = [c + 'classVec' for c in Categorical_] + Numerical_
assembler = VectorAssembler(inputCols = assemblerInputs, outputCol = 'features')
stages += [assembler]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(data)
data = pipelineModel.transform(data)
selectedCols = ['label', 'features'] + cols
data = data.select(selectedCols)
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)

In [56]:
data.toPandas().head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   label                                           features  ... TotalCharges  Churn
0    0.0  (1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...  ...      1734.65     No
1    0.0  (0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...  ...       3973.2     No
2    1.0  (0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...  ...      2869.85    Yes
3    0.0  (1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, ...  ...        238.5     No
4    0.0  (1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...  ...        119.5     No

[5 rows x 22 columns]

In [114]:
# data scale testing 
scaled_data = data
#scaled_data, x = data.randomSplit([0.75,0.25], seed = 123)
#scaled_data, x = data.randomSplit([0.50,0.50], seed = 123)
#scaled_data, x = data.randomSplit([0.25,0.75], seed = 123)

train, test = scaled_data.randomSplit([0.7,0.3], seed = 123)
print("Training Dataset :" + str(train.count()))
print("Testing Dataset :" + str(test.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training Dataset :4179
Testing Dataset :1807

In [115]:
from pyspark.ml.classification import LogisticRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter = 2)
lrModel = lr.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
lr_summary = lrModel.summary
print("area under ROC :"+str(lr_summary.areaUnderROC))
print("precision :"+str(lr_summary.precisionByLabel))
print("recall :"+str(lr_summary.recallByLabel))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

area under ROC :0.9996264008864048
precision :[0.9499072356215214, 1.0]
recall :[1.0, 0.8536585365853658]

In [118]:
pred = lrModel.transform(test)
label = pred.select('label').toPandas()
prediction = pred.select('prediction').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
#sc.install_pypi_package("sklearn")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print(str(confusion_matrix(label, prediction)))
print(str(accuracy_score(label,prediction)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[1323    4]
 [ 230  250]]
0.8705035971223022

In [121]:
from pyspark.ml.classification import DecisionTreeClassifier

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
dt_model = dt.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
predictions = dt_model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
label = predictions.select('label').toPandas()
prediction = predictions.select('prediction').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
print(str(confusion_matrix(label, prediction)))
print(str(accuracy_score(label,prediction)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[1327    0]
 [   0  480]]
1.0

In [127]:
from pyspark.ml.classification import LinearSVC
lsvc = LinearSVC(maxIter=10, regParam=0.1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
lsvcModel = lsvc.fit(train.select(['label','features']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
predictions = lsvcModel.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [135]:
label = predictions.select('label').toPandas()
prediction = predictions.select('prediction').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
print(str(confusion_matrix(label, prediction)))
print(str(accuracy_score(label,prediction)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[1320    7]
 [ 104  376]]
0.9385722191477587